In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
from datetime import datetime
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [3]:
# Input
root_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe'
ActiPAL_15s_epoch_proc_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/processed_data/ActiPAL/epoch_data/15s_ref'
ActiPAL_20hz_raw_folder = '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/activPAL_raw'

# sleep time input_data
sleep_time_file = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/metadata/record_availability_final_pal.csv'

# Output
ActiPAL_output_folder = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/white'

### Load Sleep Data

In [4]:
raw_files = [f for f in listdir(ActiPAL_20hz_raw_folder) if isfile(join(ActiPAL_20hz_raw_folder, f))]
epoch_files = [f for f in listdir(ActiPAL_15s_epoch_proc_folder) if isfile(join(ActiPAL_15s_epoch_proc_folder, f))]

data_dict = {f.split('-')[0]: {'raw': join(ActiPAL_20hz_raw_folder, f)} for f in raw_files}
for ef in epoch_files:
  key = ef.split('-')[0]
  if key in data_dict.keys():
    data_dict[key]['epoch'] = join(ActiPAL_15s_epoch_proc_folder, ef)

In [5]:
print(data_dict['OA46'])

{'raw': '/content/drive/My Drive/OA_Gihan/OA_all/OA activity data_La Trobe/activPAL_raw/OA46-AP472602 08Mar16 04-00pm for 5d 20h 14m-VANE-PB08100946-AccelDataUncompressed.csv', 'epoch': '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/Data/processed_data/ActiPAL/epoch_data/15s_ref/OA46-AP472602 08Mar16 04-00pm for 5d 20h 14m-CREA-PB08100943-15sEpochs_15sREF.csv'}


In [6]:
sleep_df = pd.read_csv(sleep_time_file)
sleep_df.head(2)

,ID,Participant,activpal_Start_time,activpal_End_time,activpal_Time_difference,actigraph_Start_time,actigraph_End_time,actigraph_Time_difference,Overlap,age_yr,sex,BMI,bodymass_kg,AG_data,AP_data,"Side_TKA (1=L,2=R)",slnumnew,nonwearmins,sleepmins,nonwearpct,justnonwear,justsleep,begin_time_pal,end_time_pal
0,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-06 14:59:46,2019-05-06 22:00:00
1,0,HOA1,2019-05-06 14:59:46,2019-05-13 08:48:55,161h 49m 9d,2019-05-06 15:00:00,2019-05-13 07:59:59,160h 59m 59d,160h 59m 59d,52.0,female,76.86261,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-05-07 06:00:00,2019-05-07 22:00:00


### Generate statistical features

In [7]:
FREQUENCY = 20 #HZ amount of the accelerometer
REFERENCE_EPOCH = 15 #Epoch value that EE is given
TIME_EPOCH_DICT = {
    # 'Epoch1': FREQUENCY * 1,
    'Epoch5': FREQUENCY * 5,
    # 'Epoch30': FREQUENCY * 30,
    # 'Epoch60': FREQUENCY * 60
}

In [8]:
def white_ee_prediction_thigh_1(row):
  predicted_ee = 5.01 + 1.000*(13.4 + 0.5674*epoch_df_current['enmo'][row.name]*1000)
  met_ee = predicted_ee/71.225
  return met_ee

def white_ee_prediction_thigh_2(row):
  predicted_ee = -10.58 + 1.1176*(13.4 + 0.5674*epoch_df_current['enmo'][row.name]*1000) + 2.9418*((13.4 + 0.5674*epoch_df_current['enmo'][row.name]*1000)**0.5) - 0.00059277*((13.4 + 0.5674*epoch_df_current['enmo'][row.name]*1000)**2)
  met_ee = predicted_ee/71.225
  return met_ee

In [10]:
for _, row in tqdm(sleep_df.iterrows(), total=sleep_df.shape[0]):
  user_id = row['Participant']
  print(_,user_id)
  # if(_<462):continue
  try:
  
    begin_time = row['begin_time_pal']
    end_time = row['end_time_pal']

    # if('HOA1' != user_id):
    #   continue
    print(user_id)

    # get respective epoch dataset
    epoch_df = pd.read_csv(data_dict[user_id]['epoch'])
    epoch_df = epoch_df[(begin_time < epoch_df['Time_stamp']) & (epoch_df['Time_stamp'] < end_time)]
    # print(list(epoch_df.index)[0])
    # continue
    # get respective raw dataset
    raw_begin_index = list(epoch_df.index)[0] * REFERENCE_EPOCH * FREQUENCY
    raw_data_length = epoch_df.shape[0] * REFERENCE_EPOCH * FREQUENCY
    raw_df_orig = pd.read_csv(data_dict[user_id]['raw'], skiprows=raw_begin_index,sep = ';', nrows=raw_data_length,usecols=[0,2,3,4], header=None)
    raw_df_orig.columns = ['Timestamp', 'X', 'Y', 'Z']
    # print(raw_df_orig.head(20))
    raw_df_orig['X_sc'] = 0.0157723997930183*raw_df_orig['X'] - 2.01571269354774
    raw_df_orig['Y_sc'] = 0.0157723997930183*raw_df_orig['Y'] - 2.01571269354774
    raw_df_orig['Z_sc'] = 0.0157723997930183*raw_df_orig['Z'] - 2.01571269354774
    # print(raw_df_orig.head(20))
    # break

    """
    Process RAW data as required by the model features
    """

    # VM and MANGLE needs for calculations by Staudenmayer model
    # Calculate the vector magnitude from X, Y, Z raw readings
    # Calculate the angle of arcsin from X and VM, arcsin(axis used/vector magnitude)/(pi/2)
    raw_df_orig['vm'] = np.sqrt([(raw_df_orig.X_sc ** 2) + (raw_df_orig.Y_sc ** 2) + (raw_df_orig.Z_sc ** 2)])[0]
    # raw_df_orig['angle'] = (90 * np.arcsin(raw_df_orig.X / raw_df_orig['vm'])) / (math.pi / 2)
    # print(raw_df_orig['vm'].head(20))
    # Calculate ENMO (Euclidean Norm Minus One) reguired for Hilderbrand model
    raw_df_orig['enmo'] = np.sqrt([(raw_df_orig.X_sc ** 2) + (raw_df_orig.Y_sc ** 2) + (raw_df_orig.Z_sc ** 2)])[0] - 1
    raw_df_orig.loc[raw_df_orig.enmo < 0, 'enmo'] = 0
    # print(raw_df_orig['enmo'].head(20))

    # For each epoch given
    for epoch_name, epoch_duration in TIME_EPOCH_DICT.items():
      print(epoch_name,epoch_duration)
      
      # continue
      # Create output path for the combined file
      output_file_name = join(ActiPAL_output_folder, epoch_name,'{}_{}_to_{}_{}.csv'.format(user_id, begin_time, end_time, epoch_name).replace(':', '-').replace(' ', '_'))
      if not exists(join(ActiPAL_output_folder, epoch_name)):
        makedirs(join(ActiPAL_output_folder, epoch_name))


      # Copy raw data
      raw_df = raw_df_orig.copy()
      # print(raw_df.head(10))
      # print(raw_df.index)
      raw_df.reset_index(drop=True, inplace=True)
      # print(raw_df.head(10))
      # print(raw_df.index)
      
      """
      Calculate the statistical inputs (Features)
      """
      # print(raw_df.index // epoch_duration)
      
      hip_grouped_temp = raw_df.groupby(raw_df.index // epoch_duration)
      
      aggregated_hip = pd.DataFrame()

      # General stats
      aggregated_hip['mvm'] = hip_grouped_temp['vm'].mean()
      aggregated_hip['enmo'] = hip_grouped_temp['enmo'].mean()
      aggregated_hip['enmo'] = aggregated_hip['enmo'].fillna(1)

    

      """
      Repeat rows of epoch file as required to supplement raw data.
      """
      # Number of rows for each epoch
      epoch_df_current = epoch_df.copy()
      # print(epoch_df_current.head())
      epoch_df_current.reset_index(drop=True, inplace=True)
      # print(epoch_df_current.head())
      
      multiplication_factor = int(REFERENCE_EPOCH / (epoch_duration/FREQUENCY))#ex: 60/(1800/60) for 30 epochs, ultimately 60/30
      # print('mul',[multiplication_factor,REFERENCE_EPOCH,epoch_duration,FREQUENCY])
      # print('before',epoch_df_current.head())
      epoch_df_current = epoch_df_current.reindex(np.repeat(epoch_df_current.index.values, multiplication_factor), method='ffill')
      # print('after',epoch_df_current.head())
      epoch_df_current['thigh_ee_final'] = epoch_df_current['thigh_EE']/multiplication_factor
      # print('EE',epoch_df_current['waist_ee'])

      epoch_df_current.reset_index(drop=True, inplace=True)
      


      epoch_df_current['enmo'] = aggregated_hip['enmo']
      epoch_df_current['mvm'] = aggregated_hip['mvm']

      epoch_df_current['EE_white_1'] = epoch_df_current.apply(white_ee_prediction_thigh_1,axis=1)
      epoch_df_current['EE_white_2'] = epoch_df_current.apply(white_ee_prediction_thigh_2,axis=1)
      

      epoch_df_current.to_csv(output_file_name, index=None)

  except Exception as e:
    f = open('errors.txt','w+')
    f.write(str(user_id)+" "+str(e)+" \n")
    f.close()

  0%|          | 0/1367 [00:00<?, ?it/s]

0 HOA1
HOA1
Epoch5 100


  0%|          | 1/1367 [00:01<24:20,  1.07s/it]

1 HOA1
HOA1
Epoch5 100


  0%|          | 2/1367 [00:03<33:30,  1.47s/it]

...

1364 OA9
OA9
Epoch5 100


100%|█████████▉| 1365/1367 [58:55<00:06,  3.08s/it]

1365 OA9
OA9
Epoch5 100


100%|██████████| 1367/1367 [59:00<00:00,  2.59s/it]

1366 OA9
OA9


In [ ]:
f_c = '/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/hibbing/Epoch1/HOA1_2019-05-06_15-00-00_to_2019-05-06_22-00-00_Epoch1.csv'
f_c_df = pd.read_csv(f_c)
f_c_df.head()


,AxisY,AxisX,AxisZ,Time_stamp,waist_vm_60,waist_vm_cpm,waist_cpm,waist_ee,waist_ee_final,enmo,mvm
0,938,169,2570,2019-05-06 15:01:00,2741.040861,2741.040861,938,3.034394,0.050573,0.051245,1.020139
1,938,169,2570,2019-05-06 15:01:00,2741.040861,2741.040861,938,3.034394,0.050573,0.046331,1.034002
2,938,169,2570,2019-05-06 15:01:00,2741.040861,2741.040861,938,3.034394,0.050573,0.078401,1.039510
3,938,169,2570,2019-05-06 15:01:00,2741.040861,2741.040861,938,3.034394,0.050573,0.051831,1.014698
4,938,169,2570,2019-05-06 15:01:00,2741.040861,2741.040861,938,3.034394,0.050573,0.067690,1.039701


In [ ]:
test = [0.051245,0.046331,0.078401,0.051831,0.067690]
t_df = pd.DataFrame(test)
s = t_df.rolling(2)
print(s.head())

AttributeError: ignored

In [ ]:
CWR_list_enmo = []
CWR_list_gvm = []
for i in range(len(f_c_df)):

  mean_w_e = f_c_df['enmo'].iloc[i:i+10].mean()
  std_w_e = f_c_df['enmo'].iloc[i:i+10].std()
  if(mean_w_e==0):
    cov_w_e = 0
  else:
    cov_w_e = (std_w_e/mean_w_e)*100
  
  
  if(math.isnan(cov_w_e)):cov_w_e = 0
  CWR_list_enmo.append(cov_w_e)

  mean_w_g = f_c_df['mvm'].iloc[i:i+10].mean()
  std_w_g = f_c_df['mvm'].iloc[i:i+10].std()
  if(mean_w_g==0):
    cov_w_g = 0
  else:
    cov_w_g = (std_w_g/mean_w_g)*100
  
  
  if(math.isnan(cov_w_g)):cov_w_g = 0
  CWR_list_gvm.append(cov_w_g)
 
# print(CWR_list_enmo[-1])

# print(CWR_list_enmo[-1])
# print(len(CWR_enmo_df),len(f_c_df))


# print(CWR_list_gvm[-1])

# print(CWR_list_gvm[-1])
# print(len(CWR_gvm_df),len(f_c_df))

0
0
25140 25140
0
0
25140 25140


In [ ]:
for i,row in f_c_df.iterrows():
  next_10_1s_enmo = []
  next_10_1s_gvm = []
  # print(row['enmo'])
  for k in range(i,i+10):
    try:
        row['enmo'].sh
        # next_10_1s_enmo.append(f_c_df['enmo'].iloc[[k]])
    except IndexError:
   
        # next_10_1s_enmo.append(0)
  enmo_df = pd.DataFrame(next_10_1s_enmo)
  print(i,next_10_1s_enmo)
  print('cv',enmo_df.cov())
  if(i==10):break
      # f.write(str(row))
      # next_itms = []
      # for k in range(i,i+10):
      #   try:
      #     # next_itms+=" "+str(epoch_df_current.iloc[[k]]['mvm'])
      #     next_itms.append(epoch_df_current['mvm'].iloc[[k]])
      #   except IndexError:
      #     next_itms+="0"
      #     next_itms.append(0)
      #   print('next',epoch_df_current.iloc[[k]]['mvm'])
      # row['next_items'] = next_itms
      # f.write(str(i)+" "+str(next_itms)+" "+str(len(next_itms)))
      # if(i==20):break
      # print('nxt',next_itms)

0 [0    0.051245
Name: enmo, dtype: float64, 1    0.046331
Name: enmo, dtype: float64, 2    0.078401
Name: enmo, dtype: float64, 3    0.051831
Name: enmo, dtype: float64, 4    0.06769
Name: enmo, dtype: float64, 5    0.053851
Name: enmo, dtype: float64, 6    0.083178
Name: enmo, dtype: float64, 7    0.04732
Name: enmo, dtype: float64, 8    0.075148
Name: enmo, dtype: float64, 9    0.064201
Name: enmo, dtype: float64]
cv     0   1   2   3   4   5   6   7   8   9
0 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
1 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
2 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
3 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
4 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
5 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
6 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
7 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
8 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
9 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN
1 [1    0.046331
Name: enmo, dtype: float64, 2    0.078401
Name: enmo, dtype: float64, 3    0.051831
Name: enmo, d

In [ ]:

f = open('/content/drive/My Drive/OA activity data_La Trobe/Analytics_Scripts/Gihan_analysis/processed data/hibbing/file.txt','w')
f.write("hi")
f.close()

In [ ]:
s = [1,2,3,2,2]
print("ss"+str(s))

ss[1, 2, 3, 2, 2]


In [ ]:
df1 = pd.DataFrame()
x = np.linspace(0,1000,100)
df1['x'] = x

y = np.linspace(0,500,100)
df1['y'] = y

z = np.linspace(0,3000,100)
df1['z'] = z

gdf = df1.groupby(df1.index//10)

for name,group in gdf:
  print(name)
  print(group)

print(gdf.groups)

AttributeError: ignored